<a href="https://colab.research.google.com/github/yyuchunn/Portfolio_Management/blob/main/2021_Portfolio_Management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 匯入函式庫

In [8]:
import numpy as np

## 設定變數

In [9]:
data_path = '/content/2021.txt'

training_range = [20150427, 20180426]
validation_range = [20180426, 20210426]

init_fund = 10e6

## 函式

### 新增元素(字典)

In [10]:
def push_dict_mapping(k: str, v: str or list, dict_: dict, mode: str = 'a'):
    if mode == 'a':
        if k not in dict_:
            dict_[k] = [v]
        else:
            dict_[k].append(v)
    else:
        if k not in dict_:
            dict_[k] = v

    return dict_


### 移動平均

In [11]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

### 取得某範圍的資料

In [12]:
def get_target_data(list_: list, range_: list, col: int = 3, prefix: int = 19):
    index_list = [0, 0]
    for item_index, item in enumerate(list_):
        if item[0] == range_[0]:
            index_list[0] = item_index - prefix
            continue
        elif item[0] == range_[1]:
            index_list[1] = item_index
            break

    return [item[col] for item in list_[index_list[0]:index_list[1] + 1]]

### 執行策略

In [13]:
def strategy_operation(data: list, buy_point: list, sell_point: list):
    buy_index = 0
    sell_index = 0

    fund = init_fund
    unit = 0

    status = 0
    if buy_point[buy_index] < sell_point[sell_index]:
        unit = fund / data[buy_point[buy_index]]
        fund = 0
        buy_index += 1
    else:
        fund += unit * data[sell_point[sell_index]]
        unit = fund / data[sell_point[sell_index]]
        sell_index += 1

        status = 1

    while True:
        if sell_index >= len(sell_point):
            if buy_index < len(buy_point):
                fund += unit * data[buy_point[buy_index]]
                unit = fund / data[buy_point[buy_index]]
                fund = 0

                status = 0
            buy_index += 1
            break
        elif buy_index >= len(buy_point):
            if sell_index < len(sell_point):
                fund += unit * data[sell_point[sell_index]]
                unit = fund / data[sell_point[sell_index]]

                status = 1
            sell_index += 1
            break
        elif buy_point[buy_index] < sell_point[sell_index]:
            if status == 1:
                fund += unit * data[buy_point[buy_index]]
                unit = fund / data[buy_point[buy_index]]
                fund = 0

                status = 0
            buy_index += 1
        elif buy_point[buy_index] > sell_point[sell_index]:
            if status == 0:
                fund += unit * data[sell_point[sell_index]]
                unit = fund / data[sell_point[sell_index]]

                status = 1
            sell_index += 1

    if unit > 0:
        fund += unit * data[-1]
    return fund

## 讀檔並載入資料

In [14]:
with open(data_path, mode='r', encoding='cp950') as f:
    title = f.readline().split('\t')

    print(title)

    stock_dict = {}
    data_dict = {}
    for row in f.readlines():
        # print(row, end='')

        row = [item.strip() for item in row.split('\t')]

        stock_dict = push_dict_mapping(row[0], row[1], stock_dict, mode='')
        temp = [float(item) for item in row[2:-3]]
        temp.append(float(row[-2]))
        data_dict = push_dict_mapping(row[0], temp, data_dict)


['證券代碼', '簡稱', '年月日', '最高價(元)', '最低價(元)', '收盤價(元)', '週轉率％', '報酬率-Ln', '開盤價(元)', '報酬率％\n']


## 顯示資料

In [15]:
print(stock_dict)

{'2303': '聯電'}


In [16]:
print(data_dict)

{'2303': [[20150105.0, 11.9655, 11.346, 11.8106, 0.9128, 11.4234], [20150106.0, 11.6945, 11.4234, 11.4234, 0.5812, 11.617], [20150107.0, 11.5783, 11.346, 11.4621, 0.3701, 11.4234], [20150108.0, 11.6945, 11.5396, 11.5396, 0.5173, 11.617], [20150109.0, 11.8494, 11.617, 11.6945, 0.7371, 11.7719], [20150112.0, 12.4689, 11.8106, 12.1979, 1.8337, 11.8106], [20150113.0, 12.3528, 12.0043, 12.043, 0.807, 12.2366], [20150114.0, 12.0043, 11.7332, 11.8106, 0.8385, 11.9655], [20150115.0, 11.9655, 11.6557, 11.6557, 0.7573, 11.8106], [20150116.0, 11.9655, 11.3847, 11.3847, 1.2706, 11.6945], [20150119.0, 11.7332, 11.5396, 11.6557, 0.6283, 11.5783], [20150120.0, 11.8494, 11.7332, 11.7719, 0.3572, 11.8106], [20150121.0, 11.8881, 11.6557, 11.7332, 0.3758, 11.8494], [20150122.0, 12.043, 11.8106, 11.9655, 0.6736, 11.8494], [20150123.0, 12.1591, 12.0043, 12.0817, 0.7514, 12.1204], [20150126.0, 12.1591, 11.9268, 12.0043, 0.4409, 12.0043], [20150127.0, 12.0817, 11.8494, 11.9655, 0.4614, 12.0817], [20150128.0,

In [17]:
print(data_dict['2303'])

[[20150105.0, 11.9655, 11.346, 11.8106, 0.9128, 11.4234], [20150106.0, 11.6945, 11.4234, 11.4234, 0.5812, 11.617], [20150107.0, 11.5783, 11.346, 11.4621, 0.3701, 11.4234], [20150108.0, 11.6945, 11.5396, 11.5396, 0.5173, 11.617], [20150109.0, 11.8494, 11.617, 11.6945, 0.7371, 11.7719], [20150112.0, 12.4689, 11.8106, 12.1979, 1.8337, 11.8106], [20150113.0, 12.3528, 12.0043, 12.043, 0.807, 12.2366], [20150114.0, 12.0043, 11.7332, 11.8106, 0.8385, 11.9655], [20150115.0, 11.9655, 11.6557, 11.6557, 0.7573, 11.8106], [20150116.0, 11.9655, 11.3847, 11.3847, 1.2706, 11.6945], [20150119.0, 11.7332, 11.5396, 11.6557, 0.6283, 11.5783], [20150120.0, 11.8494, 11.7332, 11.7719, 0.3572, 11.8106], [20150121.0, 11.8881, 11.6557, 11.7332, 0.3758, 11.8494], [20150122.0, 12.043, 11.8106, 11.9655, 0.6736, 11.8494], [20150123.0, 12.1591, 12.0043, 12.0817, 0.7514, 12.1204], [20150126.0, 12.1591, 11.9268, 12.0043, 0.4409, 12.0043], [20150127.0, 12.0817, 11.8494, 11.9655, 0.4614, 12.0817], [20150128.0, 12.1204,

In [18]:
for key, value in stock_dict.items():
    print(key, value, sep=':\t')

2303:	聯電


In [19]:
for item in data_dict['2303']:
    print(item)

[20150105.0, 11.9655, 11.346, 11.8106, 0.9128, 11.4234]
[20150106.0, 11.6945, 11.4234, 11.4234, 0.5812, 11.617]
[20150107.0, 11.5783, 11.346, 11.4621, 0.3701, 11.4234]
[20150108.0, 11.6945, 11.5396, 11.5396, 0.5173, 11.617]
[20150109.0, 11.8494, 11.617, 11.6945, 0.7371, 11.7719]
[20150112.0, 12.4689, 11.8106, 12.1979, 1.8337, 11.8106]
[20150113.0, 12.3528, 12.0043, 12.043, 0.807, 12.2366]
[20150114.0, 12.0043, 11.7332, 11.8106, 0.8385, 11.9655]
[20150115.0, 11.9655, 11.6557, 11.6557, 0.7573, 11.8106]
[20150116.0, 11.9655, 11.3847, 11.3847, 1.2706, 11.6945]
[20150119.0, 11.7332, 11.5396, 11.6557, 0.6283, 11.5783]
[20150120.0, 11.8494, 11.7332, 11.7719, 0.3572, 11.8106]
[20150121.0, 11.8881, 11.6557, 11.7332, 0.3758, 11.8494]
[20150122.0, 12.043, 11.8106, 11.9655, 0.6736, 11.8494]
[20150123.0, 12.1591, 12.0043, 12.0817, 0.7514, 12.1204]
[20150126.0, 12.1591, 11.9268, 12.0043, 0.4409, 12.0043]
[20150127.0, 12.0817, 11.8494, 11.9655, 0.4614, 12.0817]
[20150128.0, 12.1204, 11.8494, 12.1204,

In [20]:
for key in data_dict.keys():
    print(key, stock_dict[key] ,len(data_dict[key]), sep=':\t')

2303:	聯電:	1539


## 計算六年資料量

In [21]:
count = 0
for item in data_dict['2303']:
    if item[0] >= training_range[0] and item[1] <= validation_range[1]:
        count += 1
print(count)

count += 19
print(count)

1468
1487


## 訓練期

### 設定變數

In [22]:
company = '2303'

Ht = get_target_data(data_dict[company], training_range, col=1)
Lt = get_target_data(data_dict[company], training_range, col=2)
Ct = get_target_data(data_dict[company], training_range)
turnover_rate_t = get_target_data(data_dict[company], training_range, col=4)
Ot = get_target_data(data_dict[company], training_range, col=5)

H20 = moving_average(np.asarray(Ht[:-1], dtype=np.float), 20)
L20 = moving_average(np.asarray(Lt[:-1], dtype=np.float), 20)

### 找贏家點與輸家點

In [23]:
winner_index = []
loser_index = []
for index in range(len(Ct[:-1])):
    if Ct[index] >= Ht[index]:
        winner_index.append(index)
    elif Ct[index] <= Lt[index]:
        loser_index.append(index)

print(len(winner_index))
print(len(loser_index))

print(winner_index)

132
148
[2, 4, 5, 12, 17, 50, 53, 60, 66, 77, 80, 88, 91, 109, 112, 114, 120, 125, 131, 133, 134, 136, 144, 149, 159, 169, 178, 200, 206, 208, 213, 214, 228, 234, 235, 240, 247, 248, 252, 255, 257, 267, 277, 278, 279, 280, 281, 282, 283, 285, 286, 287, 288, 289, 291, 293, 294, 298, 300, 301, 308, 309, 318, 320, 323, 326, 333, 336, 338, 339, 348, 354, 356, 366, 368, 373, 374, 375, 382, 383, 385, 390, 397, 408, 414, 417, 435, 436, 441, 443, 444, 453, 458, 459, 461, 466, 492, 501, 508, 513, 519, 523, 526, 529, 534, 535, 542, 544, 545, 551, 557, 560, 565, 578, 588, 590, 600, 643, 662, 667, 685, 686, 687, 692, 714, 718, 722, 725, 728, 730, 733, 749]


### 計算處分效果

In [24]:
avg_winner_turnover_rate = np.mean(np.asarray(turnover_rate_t, dtype=np.float)[winner_index])
avg_loser_turnover_rate = np.mean(np.asarray(turnover_rate_t, dtype=np.float)[loser_index])

print(avg_winner_turnover_rate)
print(avg_loser_turnover_rate)

disposition_effect = avg_winner_turnover_rate / avg_loser_turnover_rate
print(disposition_effect)

0.29103636363636365
0.2721777027027027
1.0692880450764186


### 找出買賣點

In [25]:
buy_point1 = []
buy_point2 = []

sell_point1 = []
sell_point2 = []

for index in winner_index:
    if turnover_rate_t[index] < avg_winner_turnover_rate:
        buy_point1.append(index)
    elif turnover_rate_t[index] > avg_loser_turnover_rate:
        buy_point2.append(index)
for index in loser_index:
    if turnover_rate_t[index] > avg_winner_turnover_rate:
        sell_point1.append(index)
    elif turnover_rate_t[index] < avg_loser_turnover_rate:
        sell_point2.append(index)

print(len(buy_point1))
print(len(buy_point2))
print(len(sell_point1))
print(len(sell_point2))

95
37
52
94


### 執行四個策略，並找出最佳

In [26]:
fund1 = strategy_operation(Ot[1:], buy_point1, sell_point1)
fund2 = strategy_operation(Ot[1:], buy_point2, sell_point1)
fund3 = strategy_operation(Ot[1:], buy_point1, sell_point2)
fund4 = strategy_operation(Ot[1:], buy_point2, sell_point2)
print(fund1, fund2, fund3, fund4, sep='\t')

funds = np.asarray([fund1, fund2, fund3, fund4], dtype=np.float)
good_index = np.argmax(funds)
print(good_index)

13264516992570.744	1331942645299.2415	2.4912699547544994e+19	651585723832.4655
2


## 檢驗期

### 設定變數

In [27]:
Ht = get_target_data(data_dict[company], validation_range, col=1)
Lt = get_target_data(data_dict[company], validation_range, col=2)
Ct = get_target_data(data_dict[company], validation_range)
turnover_rate_t = get_target_data(data_dict[company], validation_range, col=4)
Ot = get_target_data(data_dict[company], validation_range, col=5)

H20 = moving_average(np.asarray(Ht[:-1], dtype=np.float), 20)
L20 = moving_average(np.asarray(Lt[:-1], dtype=np.float), 20)

### 找贏家點與輸家點

In [28]:
winner_index = []
loser_index = []
for index in range(len(Ct[:-1])):
    if Ct[index] >= Ht[index]:
        winner_index.append(index)
    elif Ct[index] <= Lt[index]:
        loser_index.append(index)

print(len(winner_index))
print(len(loser_index))

88
94


### 找出買賣點

In [29]:
buy_point1 = []
buy_point2 = []

sell_point1 = []
sell_point2 = []

for index in winner_index:
    if turnover_rate_t[index] < avg_winner_turnover_rate:
        buy_point1.append(index)
    elif turnover_rate_t[index] > avg_loser_turnover_rate:
        buy_point2.append(index)
for index in loser_index:
    if turnover_rate_t[index] > avg_winner_turnover_rate:
        sell_point1.append(index)
    elif turnover_rate_t[index] < avg_loser_turnover_rate:
        sell_point2.append(index)

print(len(buy_point1))
print(len(buy_point2))
print(len(sell_point1))
print(len(sell_point2))

32
56
60
29


### 執行四個策略，並找出最佳

In [30]:
fund1 = strategy_operation(Ot[1:], buy_point1, sell_point1)
fund2 = strategy_operation(Ot[1:], buy_point2, sell_point1)
fund3 = strategy_operation(Ot[1:], buy_point1, sell_point2)
fund4 = strategy_operation(Ot[1:], buy_point2, sell_point2)
print(fund1, fund2, fund3, fund4, sep='\t')

funds = np.asarray([fund1, fund2, fund3, fund4], dtype=np.float)
good_index = np.argmax(funds)
print(good_index)

475816696366.86163	2.742375292297024e+16	32148959523594.9	2663932118447.4526
1


## 進階

In [31]:
for company in data_dict.keys():
    if len(data_dict[company]) < 1539:
        print(company, stock_dict[company], 'can\'t be executed.', end='\n\n')
        continue
    
    print(company, stock_dict[company])
    # 訓練期
    Ht = get_target_data(data_dict[company], training_range, col=1)
    Lt = get_target_data(data_dict[company], training_range, col=2)
    Ct = get_target_data(data_dict[company], training_range)
    turnover_rate_t = get_target_data(data_dict[company], training_range, col=4)
    Ot = get_target_data(data_dict[company], training_range, col=5)

    H20 = moving_average(np.asarray(Ht[:-1], dtype=np.float), 20)
    L20 = moving_average(np.asarray(Lt[:-1], dtype=np.float), 20)

    winner_index = []
    loser_index = []
    for index in range(len(Ct[:-1])):
        if Ct[index] >= Ht[index]:
            winner_index.append(index)
        elif Ct[index] <= Lt[index]:
            loser_index.append(index)

    # print(len(winner_index))
    # print(len(loser_index))

    avg_winner_turnover_rate = np.mean(np.asarray(turnover_rate_t, dtype=np.float)[winner_index])
    avg_loser_turnover_rate = np.mean(np.asarray(turnover_rate_t, dtype=np.float)[loser_index])

    print('\tavg_winner_turnover_rate:', avg_winner_turnover_rate)
    print('\tavg_loser_turnover_rate:', avg_loser_turnover_rate)

    disposition_effect = avg_winner_turnover_rate / avg_loser_turnover_rate
    print('\tdisposition_effect', disposition_effect)

    buy_point1 = []
    buy_point2 = []

    sell_point1 = []
    sell_point2 = []

    for index in winner_index:
        if turnover_rate_t[index] < avg_winner_turnover_rate:
            buy_point1.append(index)
        elif turnover_rate_t[index] > avg_loser_turnover_rate:
            buy_point2.append(index)
    for index in loser_index:
        if turnover_rate_t[index] > avg_winner_turnover_rate:
            sell_point1.append(index)
        elif turnover_rate_t[index] < avg_loser_turnover_rate:
            sell_point2.append(index)

    # print('\t', len(buy_point1))
    # print('\t', len(buy_point2))
    # print('\t', len(sell_point1))
    # print('\t', len(sell_point2))

    stop_flag = False
    points = [len(buy_point1), len(buy_point2), len(sell_point1), len(sell_point2)]
    for point in points:
        if point == 0:
            stop_flag = True
            break

    print('\t----------------訓練期----------------')
    if stop_flag:
        print('\tXXX', points)
    else:
        fund1 = strategy_operation(Ot[1:], buy_point1, sell_point1)
        fund2 = strategy_operation(Ot[1:], buy_point2, sell_point1)
        fund3 = strategy_operation(Ot[1:], buy_point1, sell_point2)
        fund4 = strategy_operation(Ot[1:], buy_point2, sell_point2)
        print('\t', fund1, fund2, fund3, fund4)

        funds = np.asarray([fund1, fund2, fund3, fund4], dtype=np.float)
        good_index = np.argmax(funds)
        print('\t', good_index)

    # 檢驗期
    Ht = get_target_data(data_dict[company], validation_range, col=1)
    Lt = get_target_data(data_dict[company], validation_range, col=2)
    Ct = get_target_data(data_dict[company], validation_range)
    turnover_rate_t = get_target_data(data_dict[company], validation_range, col=4)
    Ot = get_target_data(data_dict[company], validation_range, col=5)

    H20 = moving_average(np.asarray(Ht[:-1], dtype=np.float), 20)
    L20 = moving_average(np.asarray(Lt[:-1], dtype=np.float), 20)

    winner_index = []
    loser_index = []
    for index in range(len(Ct[:-1])):
        if Ct[index] >= Ht[index]:
            winner_index.append(index)
        elif Ct[index] <= Lt[index]:
            loser_index.append(index)

    # print('\t', len(winner_index))
    # print('\t', len(loser_index))

    buy_point1 = []
    buy_point2 = []

    sell_point1 = []
    sell_point2 = []

    for index in winner_index:
        if turnover_rate_t[index] < avg_winner_turnover_rate:
            buy_point1.append(index)
        elif turnover_rate_t[index] > avg_loser_turnover_rate:
            buy_point2.append(index)
    for index in loser_index:
        if turnover_rate_t[index] > avg_winner_turnover_rate:
            sell_point1.append(index)
        elif turnover_rate_t[index] < avg_loser_turnover_rate:
            sell_point2.append(index)

    # print('\t', len(buy_point1))
    # print('\t', len(buy_point2))
    # print('\t', len(sell_point1))
    # print('\t', len(sell_point2))

    stop_flag = False
    points = [len(buy_point1), len(buy_point2), len(sell_point1), len(sell_point2)]
    for point in points:
        if point == 0:
            stop_flag = True
            break
    
    print('\t----------------檢驗期----------------')
    if stop_flag:
        print('\tXXX', points)
        continue

    fund1 = strategy_operation(Ot[1:], buy_point1, sell_point1)
    fund2 = strategy_operation(Ot[1:], buy_point2, sell_point1)
    fund3 = strategy_operation(Ot[1:], buy_point1, sell_point2)
    fund4 = strategy_operation(Ot[1:], buy_point2, sell_point2)
    print('\t', fund1, fund2, fund3, fund4)

    funds = np.asarray([fund1, fund2, fund3, fund4], dtype=np.float)
    good_index = np.argmax(funds)
    print('\t', good_index)
    
    print()

2303 聯電
	avg_winner_turnover_rate: 0.29103636363636365
	avg_loser_turnover_rate: 0.2721777027027027
	disposition_effect 1.0692880450764186
	----------------訓練期----------------
	 13264516992570.744 1331942645299.2415 2.4912699547544994e+19 651585723832.4655
	 2
	----------------檢驗期----------------
	 475816696366.86163 2.742375292297024e+16 32148959523594.9 2663932118447.4526
	 1

